AutoRegressive Integrated Moving Average with Exogenous variables

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [5]:
df = pd.read_csv('../../Datasets/finalized/features_missing_filledmean.csv')

df_labels = pd.read_csv('../../Datasets/finalized/label_crime_bygroup.csv')

#merge labels with features
df = df.merge(df_labels, on=['NeighbourhoodCode', 'Year', 'Month'], how='left')

In [6]:

# Load and preprocess the data
data = df  # Replace 'data.csv' with your dataset file
# Combine 'Year' and 'Month' columns to create a datetime index
data['Date'] = pd.to_datetime(data[['Year', 'Month']].assign(day=1))
data.index.freq = 'MS'  # Set to month-start frequency, adjust as per your data


data.set_index('Date', inplace=True)  # Set 'Date' as the index

data = data.sort_index()

data.index = pd.DatetimeIndex(data.index.values,
                               freq="MS")

print(data.index.inferred_freq)

# Split the data into training and testing sets
train_data = data.loc['2012-01-01':'2019-12-31']  # Training data from 2000 to 2019
test_data = data.loc['2020-01-01':'2022-12-31']  # Testing data from 2020 onwards



# Define the exogenous variables
exog_features = 'PopulationEduLow' #["NetLaborParticipation","FlexibleContracts","SelfContract","PopulationEduLow","PopulationEduMedium","PopulationEduHigh","y0-15%","y15-25%","y65-%","Peoplewithmigrationbackgroud%","Averagepeopleperhousehold","Populationdensitykm2","AverageWOZ-valueofhouses(x1000euro)","Percentuninhabited(%)","Rentalproperies(%)","Tradeandcatering%","Culture/recreationproperies%","Carsperhousehold","UrbanityLevel","Inhabitants","Benches","Lights","POI","DayC","NightC","Rain Days","Rainfall(mm)","Daylight hours"]  # Replace with your own exogenous variables
exog_train = train_data[exog_features]  # Replace with your own exogenous variables
exog_test = test_data[exog_features]

# Create and fit the SARIMAX model
order = (2, 1, 1)  # ARIMA order (p, d, q)
seasonal_order = (1, 1, 1, 12)  # Seasonal order (P, D, Q, S)
model = sm.tsa.SARIMAX(train_data['Property'], order=order, seasonal_order=seasonal_order, exog=exog_train)
model_fit = model.fit(maxiter=100)

# Make predictions on the test set
predictions = model_fit.predict(start='2020-01-01', end='2022-12-31', exog=exog_test)

# Evaluate the model
mse = ((predictions - test_data['Target_Variable']) ** 2).mean()
print("Mean Squared Error:", mse)

ValueError: Inferred frequency None from passed values does not conform to passed frequency MS